<a href="https://colab.research.google.com/github/pc-Jiang/MOR_in_control_system/blob/main/MOR_in_control_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic: Model Order Reduction in Control System
Team members:
- Pengcen Jiang (PID: A59020457)
- Yi Gu (PID: A59019647)
- Shibo Hao (PID: A59017927)

In [1]:
import torch
import time

import numpy as np
import numpy.random as nrd
import numpy.linalg as nla
import matplotlib.pyplot as plt
from scipy.stats import wishart
from copy import deepcopy

### Define a linear system and generate data

In [2]:
# define the linear system
class Linear_Dynamic_System:
    '''
    dim_x: int, dimension of inate state of the system
    dim_u: int, dimension of input
    dim_y: int, dimension of observation (output)
    sys_param: dict, parameters of the dynamic matrices and noises' variance
      x_t = Ax_{t-1} + Bu_t + epsilon
      y_t = Cx_t + eta
      sys_param['A'] - matrix on x_{t-1}
      sys_param['B'] - matrix on u_t
      sys_param['C'] - matrix on x_t
      sys_param['Q'] - variance of noise epsilon (Gaussian white noise)
      sys_param['R'] - variance of noise eta (Gaussian white noise)
    '''
    def __init__(self, dim_x, dim_u, dim_y, sys_param={}):
        self.dim_x = dim_x
        self.dim_u = dim_u
        self.dim_y = dim_y
        self.sys = self.init_coef(sys_param)
    
    def init_coef(self, sys):
        if not ('R' in sys):
            sys['R'] = wishart.rvs(20, np.eye(self.dim_y, dtype='float32')) / 10
        if not ('Q' in sys):
            sys['Q'] = wishart.rvs(200, 0.5 * np.eye(self.dim_x, dtype='float32'))
        if not ('A' in sys):
            A0 = wishart.rvs(200, 2 * np.eye(self.dim_x, dtype='float32'))
            [_, eig_vec] = nla.eig(A0)
            eig_val = np.diag(nrd.random(self.dim_x))
            # set eigenvalues no larger than 1 so that it would not blow up
            eig_val = eig_val.astype(np.float32)
            sys['A'] = eig_vec @ eig_val @ np.transpose(eig_vec)
        if not ('B' in sys): 
            sys['B'] = nrd.rand(self.dim_x, self.dim_u)
        if not ('C' in sys):
            sys['C'] = nrd.rand(self.dim_y, self.dim_x)
            sys['C'] = sys['C'].astype(np.float32)

        for k, v in sys.items():
            sys[k] = np.array(v, dtype='float32')
        return sys
    
    
    def gen(self, x0, u, length):
        '''
        Input: 
          x0 - initial state of the system, shape (dim_x, 1)
          u - stimulus of the system, shape (dim_u, length)
          length - time steps to simulate
        Output:
          x - inate state, shape (dim_x, length)
          y - observation of the system, shape (dim_y, length) 
        '''
        xt = x0
        for i in range(length):
            epsilon = nrd.multivariate_normal(np.zeros(self.dim_x), self.sys['Q'])
            epsilon = epsilon.reshape(self.dim_x, -1)
            if self.dim_y == 1:
                eta = nrd.randn(1) * self.sys['R']
            else:    
                eta = nrd.multivariate_normal(np.zeros(self.dim_y), self.sys['R'])
            eta = eta.reshape(self.dim_y, -1)
            ut = u[:, i].reshape(self.dim_u, -1)
            xt = self.sys['A'] @ xt + self.sys['B'] @ ut + epsilon
            yt = self.sys['C'] @ xt + eta
            if i == 0:
                x = xt
                y = yt
            else: 
                x = np.concatenate([x, xt], axis=1)
                y = np.concatenate([y, yt], axis=1)
        return x, y

In [3]:
# define the dimension of the system
dim_x = 50 # dimension of inate state
dim_y = 1 # dimension of output
dim_u = 5 # dimention of input
time_steps = 200 # time steps to be generated
dim_x_reduct = 5 # dimension after MOR

In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [5]:

original_sys = Linear_Dynamic_System(dim_x, dim_u, dim_y) # define a system

In [6]:


# randomly generalize the input u and x0
u = nrd.rand(dim_u, time_steps)
x0 = nrd.rand(dim_x, 1)

# generate data
x, y = original_sys.gen(x0, u, time_steps)

In [9]:
print("the shape of x: ", x.shape)

the shape of x:  (50, 200)


In [10]:
print("the shape of y: ", y.shape)

the shape of y:  (1, 200)


#### Proper orthogonal decomposition (POD)

#### Balanced Truncated (BR) Method

In [50]:
# get the parameters of the original system
sys_params = original_sys.sys

# get the controllability and observability matrices
C_matrix = sys_params['B']
O_matrix = sys_params['C']
Ct = C_matrix
Ot = O_matrix
for i in range(1, dim_x):
    Ct = sys_params['A'] @ Ct
    Ot = Ot @ sys_params['A']
    C_matrix = np.concatenate([C_matrix, Ct], axis=1) # controllability matrix
    O_matrix = np.concatenate([O_matrix, Ot], axis=0) # observability matrix

# controllability Gramian
Wc = C_matrix @ C_matrix.conjugate().transpose()
# observability Gramian
Wo = O_matrix.conjugate().transpose() @ O_matrix

# compute the truncated matrix by eigenvalue decomposition
def get_truncated_matrix_from_gramians(Wc, Wo):
    W = Wc @ Wo
    [_, Tu] = np.linalg.eig(W)
    sigma_c = np.linalg.inv(Tu) @ Wc @ np.linalg.inv(Tu).conjugate().transpose()
    sigma_o = Tu.conjugate().transpose() @ Wo @ Tu
    c_trans = np.diagonal(sigma_c)**(1/4)
    o_trans = np.diagonal(sigma_o)**(-1/4)
    sigma_s = np.diag(c_trans) @ np.diag(o_trans)
    T = Tu @ sigma_s
    return T
    
T = get_truncated_matrix_from_gramians(Wc, Wo)

In [55]:
# truncate the eigenvectors
phi = T[:, :dim_x_reduct]
psi = T.transpose()[:dim_x_reduct, :]

# set parameters of new system
sysp_br = deepcopy(sys_params)
sysp_br['A'] = psi @ sys_params['A'] @ phi
sysp_br['B'] = psi @ sys_params['B']
sysp_br['C'] = sys_params['C'] @ phi

# use the reduced order model to generate data
br_sys = Linear_Dynamic_System(dim_x_reduct, dim_u, dim_y, sysp_br)
x0_br = T.transpose() @ x0
x0_br = x0_br[:dim_x_reduct]
x_br, y_br = original_sys.gen(x0, u, time_steps)

<ipython-input-45-f3c97c62026b>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  sys[k] = np.array(v, dtype='float32')


#### Autoencoder